In [186]:
import os
from pathlib import Path
from dataclasses import dataclass
from configparser import ConfigParser
import yaml
import re
import pandas as pd
from tqdm.auto import tqdm
from datetime import datetime

## config

All the config is saved in a config.ini file

In [103]:
c = ConfigParser()
c.read("config.ini")
for section in c.sections():
    print(section)
    print(c.items(section))

blog
[('content', 'notes')]


In [69]:
path = Path(c["blog"]["content"])
path

PosixPath('notes')

('---', '\ntitle:')

In [100]:
yaml.load(txt[:112+3])

{'title': 'Testing note in a subfolder',
 'date': datetime.date(2020, 6, 14),
 'lastmod': '2019-08-7',
 'tags': ['python'],
 'draft': False,
 'toc': True}

In [113]:
n = txt[3:].find("---") + 3 + 3
txt[n:1000].strip()

'This note is inside a subfolder.\n\n## \n\n## twitter embed\n\nThis should show a twitter tweet embedded inside the page:\n\n<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Sunsets don&#39;t get much better than this one over <a href="https://twitter.com/GrandTetonNPS?ref_src=twsrc%5Etfw">@GrandTetonNPS</a>. <a href="https://twitter.com/hashtag/nature?src=hash&amp;ref_src=twsrc%5Etfw">#nature</a> <a href="https://twitter.com/hashtag/sunset?src=hash&amp;ref_src=twsrc%5Etfw">#sunset</a> <a href="http://t.co/YuKy2rcjyU">pic.twitter.com/YuKy2rcjyU</a></p>&mdash; US Department of the Interior (@Interior) <a href="https://twitter.com/Interior/status/463440424141459456?ref_src=twsrc%5Etfw">May 5, 2014</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>\n\nThe code used\n\n```html\n<blockquote class="twitter-tweet"><p lang="en" d'

In [104]:
front_matter = yaml.load(txt[:txt[3:].find("---") + 3])
front_matter

{'title': 'Testing note in a subfolder',
 'date': datetime.date(2020, 6, 14),
 'lastmod': '2019-08-7',
 'tags': ['python'],
 'draft': False,
 'toc': True}

## Read each file and parse front matter

In [188]:
ctime = p.stat().st_ctime

In [197]:
datetime.resolution()

datetime.timedelta(microseconds=1)

In [193]:
dt = datetime.fromtimestamp(ctime)
datetime(dt.year, dt.month, dt.day)

datetime.datetime(2020, 6, 17, 0, 0)

In [204]:
md_paths = [f for f in path.rglob("*.md")]

data = []

for p in md_paths:
    d = {}
    all_txt = p.read_text() 
    n = all_txt[3:].find("---") + 3
    fm = yaml.load(all_txt[:n])
    txt = all_txt[n+3:].strip()


    d["title"] = fm["title"]

    # add created date
    try:
        d["date"] = fm["date"]
    except:
        dt = datetime.fromtimestamp(p.stat().st_ctime) # create time of file
        d["date"] = dt

    # add last modified data
    try:
        d["lastmod"] = fm["lastmod"]
    except:
        dt = datetime.fromtimestamp(p.stat().st_mtime)
        d["lastmod"] = dt
    
    d["path"] = p
    d["filename"] = p.name
    d["front_matter"] = front_matter
    d["markdown"] = txt
    d["tags"] = fm["tags"] 
    data.append(d)

df = pd.DataFrame(data) #.set_index("filename")
df

,title,date,lastmod,path,filename,front_matter,markdown,tags
0,Visual Studio Code,2019-10-10 00:00:00.000,2019-10-10 00:00:00.000,notes/vs_code.md,vs_code.md,"{'title': 'Testing note in a subfolder', 'date...","VS Code has so many extensions and stuff, that...",[apps]
1,Setting up Linux on a Chromebook with Crostini,2018-08-24 00:00:00.000,2019-08-07 00:00:00.000,notes/setup-crostini-chromebook.md,setup-crostini-chromebook.md,"{'title': 'Testing note in a subfolder', 'date...",ChromeOS supports a built in Linux - running i...,[chromebooks]
2,Using Anaconda,2020-06-17 10:05:57.290,2020-06-17 10:05:57.290,notes/anaconda.md,anaconda.md,"{'title': 'Testing note in a subfolder', 'date...",## Install Anaconda\n\nThere seems to be two m...,[python]
3,Jupyter Lab tips and tricks,2019-08-04 00:00:00.000,2020-06-17 10:06:10.030,notes/jupyterlab.md,jupyterlab.md,"{'title': 'Testing note in a subfolder', 'date...",My collection of tips and tricks for using jup...,[jupyter]
4,Setting up my own blog,2020-06-14 00:00:00.000,2019-08-07 00:00:00.000,notes/building_this_blog.md,building_this_blog.md,"{'title': 'Testing note in a subfolder', 'date...",Hereby I talk through writing my own blog engi...,[python]
5,Testing note in a subfolder,2020-06-14 00:00:00.000,2019-08-07 00:00:00.000,notes/sub_notes/note_in_subfolder.md,note_in_subfolder.md,"{'title': 'Testing note in a subfolder', 'date...",This note is inside a subfolder.\n\n## twitter...,[python]


In [180]:
fm = df.front_matter[0]
fm

{'title': 'Testing note in a subfolder',
 'date': datetime.date(2020, 6, 14),
 'lastmod': '2019-08-7',
 'tags': ['python'],
 'draft': False,
 'toc': True}